In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

# Cargar el conjunto de datos
df = pd.read_csv("../data/processed/clientes_banco.csv")

# extraigo la variable a predecir
x = df.drop('permanencia', axis=1) 
y = df['permanencia']  


# codificar variables categóricas
variables_categoricas = ["genero", "nivel_educativo", "estado_civil", "banda_salarial", "categoria_tarjeta"]
codificacion = OneHotEncoder(handle_unknown='ignore')
x_codificadas = pd.DataFrame(codificacion.fit_transform(x[variables_categoricas]).toarray(), columns=codificacion.get_feature_names(variables_categoricas))
x = x.drop(variables_categoricas, axis=1)
x = pd.concat([x, x_codificadas], axis=1)
x

,edad,num_personas_dependientes,num_meses_cliente,productos_financieros_contratados,num_meses_inactivo_12_meses,veces_contactado_12_meses,limite_credito,saldo_pendiente_tarjeta,credito_disponible,dif_cantidad_gastada_trimestre,...,banda_salarial_$120K +,banda_salarial_$40K - $60K,banda_salarial_$60K - $80K,banda_salarial_$80K - $120K,banda_salarial_Less than $40K,banda_salarial_Unknown,categoria_tarjeta_Blue,categoria_tarjeta_Gold,categoria_tarjeta_Platinum,categoria_tarjeta_Silver
0,49,3,43,4,4,0,23032.0,1960,21072.0,0.619,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,41,3,34,5,3,3,34516.0,2053,32463.0,1.034,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,53,3,34,6,2,2,2940.0,1264,1676.0,0.562,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,44,2,32,3,3,3,3031.0,1879,1152.0,0.788,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,44,3,25,3,3,0,1862.0,0,1862.0,0.883,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10022,50,2,40,3,2,3,4003.0,1851,2152.0,0.703,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10023,41,2,25,4,2,3,4277.0,2186,2091.0,0.804,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10024,44,1,36,5,3,4,5409.0,0,5409.0,0.819,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
10025,30,2,36,4,3,3,5281.0,0,5281.0,0.535,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
# Construir el modelo de Random Forest
modelo_random_forest = RandomForestClassifier(n_estimators=100, random_state=1)

# Ajustar el modelo al conjunto de datos de entrenamiento
modelo_random_forest.fit(x_train, y_train)

# Hacer predicciones en el conjunto de datos de prueba
y_pred = modelo_random_forest.predict(x_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.9571286141575274


In [9]:
#PREDICCION REAL
df_datos_para_predecir = pd.read_csv("../data/processed/clientes_banco.csv")

x = df_datos_para_predecir.drop("permanencia", axis=1)
y = df_datos_para_predecir["permanencia"]

# codificar variables categóricas
variables_categoricas = ["genero", "nivel_educativo", "estado_civil", "banda_salarial", "categoria_tarjeta"]
codificacion = OneHotEncoder(handle_unknown='ignore')
x_codificadas = pd.DataFrame(codificacion.fit_transform(x[variables_categoricas]).toarray(), columns=codificacion.get_feature_names(variables_categoricas))
x = x.drop(variables_categoricas, axis=1)
x = pd.concat([x, x_codificadas], axis=1)

nueva_prediccion = modelo_random_forest.predict(x)

#permite ver mas resultados en el print
np.set_printoptions(threshold=np.inf)


print(nueva_prediccion)

['Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Attrited Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Attrited Customer' 'Existing Customer'
 'Existing Customer' 'Existing Customer' 'Attrited Customer'
 'Attrited Customer' 'Existing Customer' 'Existing Customer'
 'Existing Customer' 'Ex